In [1]:
import numpy as np
import xarray as xr
import salishsea_tools.river_202108 as rivers
import pandas as pd

In [2]:
def make_ts_arr(jdepth, data, volume_dims, rho_inv):  # to do xarray multiplication, make sure rename the dims and pass into function
    # rho = 1026  ## do 1/rho and pass it into it
    # volume_dims = np.array([volume.to_numpy(), volume.to_numpy()])[0:1] ## do this once, pass volume dims into it
    data_np = data[:, 0:jdepth].values
    river_arr = rho_inv * np.multiply(volume_dims[:, 0:jdepth], data_np) ## try again as the xarrays? and multiply by tmask so no need nansum (can acutally do that to volume)
    river_sum = np.nansum(np.nansum(np.nansum(river_arr, axis=3), axis=2), axis=1)
    return river_sum

In [3]:
def custom_arange(start, stop, step):
    num_steps = int(np.ceil((stop - start) / step))
    end_value = np.array([stop])
    ranges = np.arange(start, num_steps*step, step)
    return np.concatenate((ranges, end_value))

In [10]:
ds = xr.open_dataset('/ocean/cdonaldson/research_runs/long_dye_tracers/extractions/SalishSeaCast_river_trace_day_avg_Fraser_all_20170101_20191203.nc')
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
# the mask is 1 where there is water, we want the opposite.  The meshmask has an extra dimension, hence the [0]
tmask = 1 - mesh.tmask[0]

In [11]:
volume = mesh.e3t_0[0] * mesh.e1t[0] * mesh.e2t[0] * (1-tmask[0])
volume_dims = np.array([volume.to_numpy(), volume.to_numpy()])[0:1] ## do this once, pass volume dims into it

rho_inv = (1/1026)

In [12]:
dep_idx = 40
data = ds.fraser_river
step_len = 50
start = 0
time_idxs = custom_arange(start, len(data), step_len)
river_arr = np.zeros(len(data))

for i in np.arange(0, len(time_idxs)-1):
    print('{},{}'.format(time_idxs[i], time_idxs[i+1]))
    river_arr[time_idxs[i]:time_idxs[i+1]] = make_ts_arr(dep_idx, ds.fraser_river[time_idxs[i]:time_idxs[i+1]], volume_dims, rho_inv)

0,50
50,100
100,150
150,200
200,250
250,300
300,350
350,400
400,450
450,500
500,550
550,600
600,650
650,700
700,750
750,800
800,850
850,900
900,950
950,1000
1000,1050
1050,1067


In [13]:
variable = 'fraser_river'

filename = '1d_volume_{}_s2017_{}_dep_0_{}.csv'.format(variable, len(data), dep_idx)
pd.DataFrame(river_arr).to_csv('dye_to_vol/' + filename)
print(filename)


1d_volume_fraser_river_s2017_1067_dep_0_40.csv
